<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [14]:
import json
import logging
import random
import time
from spire.doc import *
from spire.doc.common import *
import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *
import datetime
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [15]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [16]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [17]:
vocab_dict

'はってん,いまに,くだ,ちのう,わん,ぜん,ここ,けんこう,ほこり,かいけい,ちがい,ようするに,どうとく,かたがた,さいしゅう,かたい,われわれ,みやこ,いんよう,ちょうし,てきど,ひとこと,かく,とら,きまり,てつ,いとこ,あいず,けん,たんじゅん,とざん,がら,ねんだい,つかれ,から,しょうたい,はかる,ゆうり,ば,きにいる,はたけ,きゅうりょう,たいしょう,さんか,とおりすぎる,いわい,つい,こや,おひる,ちらす,けん,ようき,そうち,あく,とし,うる,いんさつ,じょうきょう,せんでん,ちか,のんびり,てき,さく,ざせき,げんかい,はやる,たから,おさめる,まえもって,そんちょう,かく,できごと,まもる,たいほ,しょり,しゅよう,ねがい,きゅうけい,さいのう,せっきょくてき,みおくり,よき,せきゆ,こきょう,けいき,めったに,かわいらしい,めうえ,きょく,いじ,ある,ぶんや,くせ,ふく,ひとり,せいり,あらゆる,しつど,うし,ねがう,たいよう,おもいで,しょうがい,さべつ,きゅうげき,ぜんたい,いち,きかん,あわせる,かっき,うれる,さる,じさつ,わく,ゆうしょう,ことわる,じゅうし,しばふ,すえ,かげ,いねむり,やぶる,かんれん,ぜんしゃ,やく,せいけつ,そうぞう,もくてき,はんだん,せいげん,こおる,たまる,め,うさぎ,うわさ,おめでとう,あいて,ろんぶん,いっぽう,めんきょ,みる,うったえる,まよう,ゆうのう,ほう,のうぎょう,きんにく,きしゃ,さける,せんたく,み,かいふく,もくひょう,そなえる,はじまり,あら,ほお,そうじゅう,ふれる,なにか,なん,きほん,そんけい,しんぱん,しょうご,ちょうせん,じじょう,せんこう,くわえる,くみ,ねっちゅう,こうさい,まったく,すむ,はく,あてる,のう,いわ,かわ,たいがく,つなぐ,ちしき,かんそう,おる,ふしぎ,はい,さす,おさめる,おう,ほぞん,いしき,こうせい,へいきん,せんこう,くんれん,ざいさん,ひょうじょう,くつう,せいき,ころぶ,ついに,いわば,いったい,しゅつじょう,いらい,きのう,ほほ,きにゅう,オフィス,しぜん,きちょう,はね,きよう,たび,けいやく,ほんと,あい,りそう,そうおん,きり,がいしゅつ,ご,ちこく,かつやく,きゅうに,ぶたい,せんじつ,けいこう,せいきゅう,しょう,にえる,い

#### load Models

In [18]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_kanji): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

# '''
# - 問題1 のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading): 1 question
# - 問題2 このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_kanji): 1 question
# - 問題3（　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。 (word_meaning): 1 question
# - 問題4 に意味が最も近いものを、１・２・３・４から一つえらびなさい。(synonym_substitution) 1 question
# - 問題5 つぎのことばの使い方として最もよいものを、２・３・４から一つえらびなさい。 (word_usage): 1 question
# - 問題6 つぎの文の（　　　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。(sentence_grammar) 1 questions
# - 問題7 つぎの文の ★ に入る最もよいものを、１・２・３・４から一つえらびなさい。(sentence_sort) 1 question
# '''

instruction = """
Section 1: Vocabulary and Grammar
- 問題1 のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading) 8 questions in total: 3 are nouns, 3 are verbs, 1 is an adjective, and 1 is an adverb
- 問題2 このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_kanji) 6 questions in total: 2 nouns, 2 verbs, 1 adjective, and 1 adverb.
- 問題3（　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。 (word_meaning) 11 questions in total: 4 are nouns, 4 are verbs, 2 are adjectives, and 1 is an adverb.
- 問題4 に意味が最も近いものを、１・２・３・４から一つえらびなさい。(synonym_substitution) 5 questions in total: 2 are nouns, 2 are verbs, and 1 is an adjective.
- 問題5 つぎのことばの使い方として最もよいものを、２・３・４から一つえらびなさい。 (word_usage) 5 questions in total: 1 noun, 2 verbs, 1 adjective, and 1 adverb.
- 問題6 つぎの文の（　　　）に入れるのに最もよいものを、１・２・３・４から一つえらびなさい。(sentence_grammar) 13 questions in total: 5 focus on verbs, 2 on adjectives, 2 on adverbs, and 4 on particles or conjunctions.
- 問題7 つぎの文の ★ に入る最もよいものを、１・２・３・４から一つえらびなさい。(sentence_sort) 5 questions in total: 2 involve verbs, 1 noun, 1 adjective, and 1 particle or conjunction.
- 問題8 つぎの文章を読んで、文章全体の内容を考えて、文中の 19 から 22 の中に入る最もよいものを、１・２・３・４から一つえらびなさい (sentence_structure) 1 question

Section 2: Reading Comprehension
- 問題9 つぎの(1)から(4)の文章を読んで、質問に答えなさい。答えは、1・2・3・4から最もよいものを一つえらびなさい (short_passage_read): 4 articles
- Information retrieval (info_retrieval): 1 article

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 1 question

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The topic word in outline must be presented in Japanese kana only. The topic word cannot be chosen twice. The complexity should be restricted to N3 level. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of moderate and very difficult topics to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First, evenly pick Japanese words used in different part of sentence from 'Vocabulary' for questions in Vocabulary and Grammar, ensuring 80 percent of very difficult words for topics. At Section 2 and 3, randomly choose topics in 'TopicList' for reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [19]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [20]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [21]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-06-13 19:05:40,554 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [22]:
initial_outline

Outline(page_title='日本語能力試験N3 模擬試験 問題用紙', sections=[Section(section_title='第1部 語彙・文法', subsections=[Subsection(subsection_title='kanji_reading', description='8問：名詞3問、動詞3問、形容詞1問、副詞1問。', question_topics=[QuestionTopic(topic='さいのう'), QuestionTopic(topic='くわえる'), QuestionTopic(topic='はたけ'), QuestionTopic(topic='とおりすぎる'), QuestionTopic(topic='はかる'), QuestionTopic(topic='うれる'), QuestionTopic(topic='せっきょくてき'), QuestionTopic(topic='まえもって')]), Subsection(subsection_title='write_kanji', description='6問：名詞2問、動詞2問、形容詞1問、副詞1問。', question_topics=[QuestionTopic(topic='ちしき'), QuestionTopic(topic='きゅうりょう'), QuestionTopic(topic='さそう'), QuestionTopic(topic='ふれる'), QuestionTopic(topic='せいけつ'), QuestionTopic(topic='のんびり')]), Subsection(subsection_title='word_meaning', description='11問：名詞4問、動詞4問、形容詞2問、副詞1問。', question_topics=[QuestionTopic(topic='ざいさん'), QuestionTopic(topic='しゅよう'), QuestionTopic(topic='きょうかしょ'), QuestionTopic(topic='かんれん'), QuestionTopic(topic='やぶる'), QuestionTopic(topic='みおくり'), QuestionT

In [23]:
from IPython.display import display, Markdown, HTML
display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題用紙

## 第1部 語彙・文法

### kanji_reading

8問：名詞3問、動詞3問、形容詞1問、副詞1問。

- **さいのう**
- **くわえる**
- **はたけ**
- **とおりすぎる**
- **はかる**
- **うれる**
- **せっきょくてき**
- **まえもって**

### write_kanji

6問：名詞2問、動詞2問、形容詞1問、副詞1問。

- **ちしき**
- **きゅうりょう**
- **さそう**
- **ふれる**
- **せいけつ**
- **のんびり**

### word_meaning

11問：名詞4問、動詞4問、形容詞2問、副詞1問。

- **ざいさん**
- **しゅよう**
- **きょうかしょ**
- **かんれん**
- **やぶる**
- **みおくり**
- **もとめる**
- **ちらす**
- **あらた**
- **かわいらしい**
- **きゅうに**

### synonym_substitution

5問：名詞2問、動詞2問、形容詞1問。

- **せいり**
- **じょうきょう**
- **さける**
- **きにいる**
- **たんじゅん**

### word_usage

5問：名詞1問、動詞2問、形容詞1問、副詞1問。

- **かいけい**
- **しゅつじょう**
- **まよう**
- **かたい**
- **つい**

### sentence_grammar

13問：動詞5問、形容詞2問、副詞2問、助詞・接続詞4問。

- **まもる**
- **うつす**
- **あたためる**
- **かざり**
- **まけ**
- **きちょう**
- **あきらか**
- **まったく**
- **あらゆる**
- **けれども**
- **しかし**
- **それで**
- **また**

### sentence_sort

5問：動詞2問、名詞1問、形容詞1問、助詞・接続詞1問。

- **つなぐ**
- **しゅうかく**
- **きほん**
- **ゆうり**
- **ので**

### sentence_structure

1問：文章全体の内容を考えて空欄に最もよいものを選ぶ。

- **じょうきょう**

## 第2部 読解

### short_passage_read

4問：短文読解。各文章のテーマは下記の通り。

- **購入したい商品の説明**
- **道を尋ねる**
- **週末の予定について話す**
- **健康とフィットネスについて話す**

### info_retrieval

1問：情報検索。テーマは下記の通り。

- **公共施設の利用方法について話す**

## 第3部 聴解

### topic_understanding

1問：話の主題を理解する問題。テーマは下記の通り。

- **友人との日常会話**

In [24]:
import re
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = re.findall(r'\((.*?)\)', subsection['subsection_title'])[0]

        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing 第1部 語彙・文法:   0%|          | 0/8 [00:00<?, ?it/s]


IndexError: list index out of range

In [ ]:
# yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# # Display as formatted YAML
# display(Markdown(yaml_str))

In [ ]:
output_data

In [ ]:
def render_to_html(sections):
    html = '<html><body style="text-align: left;">\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for idx, qtopic in enumerate(subsection.get("question_topics", []), 1):
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += '<div>\n'
                        # html += f'<div style="display: flex; align-items: center;"><div>{idx}.  </div>{result["html_question"]}</div>\n'
                        html += f'<p><strong>{idx}. </strong>{result["html_question"]}</p>'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li>{idx}. <b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{idx}. {choice}</li>\n"
                            html += '</ul>\n'
                        html += '</div>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["html_question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["html_question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [ ]:
html_output = render_to_html(output_data['sections'])
display(HTML(html_output))

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_folder = "output"
html_filename = f"JLPT_{timestamp}.html"
html_filepath = os.path.join(output_folder,html_filename)
with open(html_filepath, "w", encoding="utf-8") as file:
    file.write(html_output)

In [ ]:
# Create a Document object
document = Document()
# Add a section to the document
section = document.AddSection()
# Set the page margins to 72 points (72 points = 1 inch)
section.PageSetup.Margins.All = 72

# Add a paragraph to the section
paragraph = section.AddParagraph()

# Add the HTML string to the paragraph
paragraph.AppendHTML(html_output)

doc_filename = f"JLPT_{timestamp}.docx"
doc_filepath = os.path.join(output_folder,doc_filename)
# Save the result document to a DOCX file
document.SaveToFile(doc_filepath, FileFormat.Docx2016)
# Or save the result document to a DOC file
# document.SaveToFile("HtmlStringToDoc.doc", FileFormat.Doc)

document.Close()

In [ ]:
import paramiko

# Load the private key from the environment variable
# Parse the private key
key_file_path = 'temp_id_ed25519'
pkey = paramiko.Ed25519Key.from_private_key_file(key_file_path)

# SSH and SFTP parameters
hostname = "58.246.203.10"
username = "hwu"
port = 22

remote_html_file_path = f"/var/www/html/{html_filename}"
remote_doc_file_path = f"/var/www/html/{doc_filename}"

# Establish SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(hostname=hostname, port=port, username=username, pkey=pkey)

# Transfer the file
sftp = ssh.open_sftp()
sftp.put(html_filepath, remote_html_file_path)
print(f"File transferred to {hostname}:{remote_html_file_path}")
sftp.put(doc_filepath, remote_doc_file_path)
print(f"File transferred to {hostname}:{remote_doc_file_path}")

# Clean up
sftp.close()
ssh.close()